In [ ]:
import sqlite3
from io import BytesIO

import numpy as np
from PIL import Image
from keras import Input


def load_data_from_db(test, target_size=(128, 128)):
    conn = sqlite3.connect('images.db')
    cursor = conn.cursor()
    cursor.execute("SELECT imagine, sepia FROM Imagini WHERE test = ?", (True if test == 1 else False,))
    rows = cursor.fetchall()
    conn.close()

    x = []
    y = []
    for row in rows:
        imagine_blob, sepia = row
        imagine_bytes = BytesIO(imagine_blob)
        imagine = Image.open(imagine_bytes)

        if imagine.mode == "P":
            imagine = imagine.convert("RGBA")
        if imagine.mode != 'RGB':
            imagine = imagine.convert('RGB')

        imagine = imagine.resize(target_size)

        imagine = np.array(imagine) / 255.0

        x.append(imagine)
        y.append(1 if sepia else 0)

    x = np.array(x)
    y = np.array(y)
    return x, y


In [ ]:
train_input, train_output = load_data_from_db(0)
test_input, test_output = load_data_from_db(1)

In [ ]:
from keras.src.optimizers import Adam

input_shape = train_input.shape[1:]

from tensorflow.keras import layers, models

model = models.Sequential([
    Input(input_shape),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision', 'recall'])



model.fit(train_input, train_output, epochs=10)

predicted = model.predict(test_input)
predicted = [1 if p > 0.5 else 0 for p in predicted]

In [ ]:
model.summary()

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(test_output, predicted)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=["normal", "sepia"], yticklabels=["normal", "sepia"])
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
model2 = models.Sequential([
    Input(input_shape),
    layers.Conv2D(64, (4, 4), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
model2.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy', 'precision', 'recall'])

model2.fit(train_input, train_output, epochs=10)

predicted = model2.predict(test_input)
predicted = [1 if p > 0.5 else 0 for p in predicted]

In [ ]:
model2.summary()

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(test_output, predicted)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=["normal", "sepia"], yticklabels=["normal", "sepia"])
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()